# 기울기 자동 계산
- 파이토치 텐서는 자신이 어디서부터 왔는지, 즉 어느 연산을 수행해서 만들어졌는지 알 수 있기 때문에 미분을 최초 입력까지 연쇄적으로 적용해 올라갈 수 있음

In [ ]:
import numpy as np
import torch
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [ ]:
def model(t_u, w, b):
    return w * t_u + b


def loss_fn(t_p, t_u):
    squared_diffs = (t_p - t_u) ** 2
    return squared_diffs.mean()

In [ ]:
params = torch.tensor([1.0, 0.0] , requires_grad = True) 
#requires_grad params를 조상으로 두는 모든 텐서는 params에 접근할 수 있도록 허용

print(params.grad is None)
print(params.grad)


### 역전파
- .backward()를 통하여 loss를 W,b 에 대하여 각각 미분한다 => 기울기 구하기
    - backward()는 미분을 누적시키는 것이지 저장하는 것이 아니다.
    - 따라서 파라미터 조정을 위해 사용한 후에는 기울기를 0으로 초기화 할 필요가 있다
    - backward가 호출되면 손실이 다시 계산되고 backward가 다시 호출되고 각 말단 노드의 기울기 값이 기존 값에 누적되어 부정확한 기울기 값을 초래하게 된다.
    - 이를 방지하기 위하여 각 반복문에서 명시적으로 기울기를 0으로 초기화 해야 한다

In [ ]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

params.grad

- 명시적 기울기 초기화

In [ ]:
if params.grad is not None:
    params.grad.zero_()

## 자동 미분을 적용한 코드 
- 5-1장에서는 grad_fn 함수를 생성하여 수동적으로 마분을 구했지만
- .backward()를 사용하여 자동 미분을 적용했다

In [ ]:
def training_loop(n_epochs , learning_rate , params, t_u, t_c):
    for epoch in range(1,n_epochs+1):
        if params.grad is not None:
            params.grad.zero_()

        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()

        with torch.no_grad():
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print(f"Epoch : {epoch}  , Loss : {loss} , Grad  : {params.grad}")

    return params


In [ ]:
t_un = t_u * 0.1


In [ ]:
training_loop(
    n_epochs = 5000 ,
    learning_rate = 1e-2 , 
    params = torch.tensor([1.0 ,0.0] , requires_grad= True), 
    t_u = t_un, 
    t_c = t_c)